In [1]:
from scipy.sparse import csr_matrix
from scipy.sparse import coo_matrix
import numpy as np 
import torch
import dgl
from dgl.data import AsNodePredDataset
from ogb.nodeproppred import DglNodePropPredDataset

/home/bear/miniconda3/envs/dgl_v09/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
id = np.fromfile("/home/bear/workspace/single-gnn/data/partition/PD/processed/part3/trainIds.bin",dtype=np.int32)
id

array([    0,     1,     2, ..., 49150, 49151, 49152], dtype=int32)

In [12]:
labels = np.fromfile("/home/bear/workspace/single-gnn/data/partition/PD/processed/part3/labels.bin",dtype=np.int64)
labels[id].sum()

297738

In [23]:
dataset = AsNodePredDataset(DglNodePropPredDataset('ogbn-products',root="/home/bear/workspace/single-gnn/data/dataset"))
g = dataset[0]
label = g.ndata['label']
trainIdx = g.ndata['train_mask']
trainIdx = torch.nonzero(trainIdx).reshape(-1)
trainLabel = label[trainIdx]
rightSUM = trainLabel.sum()

In [2]:
trainId = torch.arange(196615,dtype=torch.int64)
batch_size = 4
trainBatch = torch.chunk(trainId, batch_size, dim=0)
maxID=2449029
GRAPHPATH="/home/bear/workspace/single-gnn/data/raid/ogbn_products"
graph = np.fromfile(GRAPHPATH+"/graph.bin",dtype=np.int32)

In [3]:
dst = torch.tensor(graph[::2])
src = torch.tensor(graph[1::2])
nodeTable = torch.zeros(maxID,dtype=torch.int32)
nodeTable[trainId] = 1
batch_size = 3
src_batches = torch.chunk(src, batch_size, dim=0)
dst_batches = torch.chunk(dst, batch_size, dim=0)
batch = [src_batches, dst_batches]
edgeTable = torch.zeros_like(src,dtype=torch.int32).cuda()
edgeNUM = 0
allEdgeNUM = src.numel()

In [4]:
import copy
repeats = 3
for index in range(1,repeats+1):
    acc_tabel = torch.zeros_like(nodeTable,dtype=torch.int32)
    #raw_nodeTabel = copy.deepcopy(nodeTable)
    print(f"before {index} BFS has {torch.nonzero(nodeTable).size(0)} nodes, "
        f"{torch.nonzero(nodeTable).size(0) * 1.0 / maxID * 100 :.2f}% of total nodes")
    for src_batch,dst_batch in zip(*batch):
        tmp_nodeTabel = copy.deepcopy(nodeTable)
        tmp_nodeTabel = tmp_nodeTabel.cuda()
        src_batch = src_batch.cuda()
        dst_batch = dst_batch.cuda()
        #dgl.fastFindNeighbor(tmp_nodeTabel, src_batch, dst_batch, acc)
        dgl.fastFindNeigEdge(tmp_nodeTabel,edgeTable,src_batch, dst_batch)
        tmp_nodeTabel = tmp_nodeTabel.cpu()
        acc_tabel = acc_tabel + tmp_nodeTabel - nodeTable
    print("end bfs...")
    edgeNUM = edgeTable.cpu().sum() - edgeNUM
    print(f"edge add to subG : {edgeNUM} , {edgeNUM * 1.0 / allEdgeNUM * 100 :.2f}% of total edges")
    nodeTable = nodeTable + acc_tabel
    print(f"after {index} BFS has {torch.nonzero(nodeTable).size(0)} nodes, "
            f"{torch.nonzero(nodeTable).size(0) * 1.0 / maxID * 100 :.2f}% of total nodes")
    print('-'*10)
edgeTable = edgeTable.cpu()

before 1 BFS has 196615 nodes, 8.03% of total nodes
end bfs...
edge add to subG : 26927735 , 21.77% of total edges
after 1 BFS has 1872007 nodes, 76.44% of total nodes
----------
before 2 BFS has 1872007 nodes, 76.44% of total nodes
end bfs...
edge add to subG : 8969549 , 7.25% of total edges
after 2 BFS has 2242369 nodes, 91.56% of total nodes
----------
before 3 BFS has 2242369 nodes, 91.56% of total nodes
end bfs...
edge add to subG : 27177668 , 21.97% of total edges
after 3 BFS has 2281666 nodes, 93.17% of total nodes
----------


In [5]:
dst_batch = dst_batch.cpu()
src_batch = src_batch.cpu()

In [6]:
edgeTable.shape

torch.Size([123718280])

In [7]:
edgeid = torch.nonzero(edgeTable).reshape(-1)

In [8]:
edgeid

tensor([       0,        1,        2,  ..., 41239424, 41239425, 41239426])

In [9]:
hop1 = graph.reshape(-1,2)[edgeid]

In [10]:
selfLoop = np.repeat(trainBatch[0], 2).reshape(-1,2)
selfLoop

tensor([[    0,     0],
        [    1,     1],
        [    2,     2],
        ...,
        [49151, 49151],
        [49152, 49152],
        [49153, 49153]])

In [13]:
selfLoop = torch.tensor(selfLoop)
hop1 = torch.tensor(hop1)
hop1 = torch.cat([selfLoop,hop1.reshape(-1,2)])
hop1

/tmp/ipykernel_1980112/829579371.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  selfLoop = torch.tensor(selfLoop)


tensor([[      0,       0],
        [      1,       1],
        [      2,       2],
        ...,
        [ 523322, 1561582],
        [1561582,  523322],
        [ 523322, 1947756]])

In [14]:
hop1 = hop1.reshape(-1)
srcs = hop1[::2]
dsts = hop1[1::2]

In [16]:
dsts

tensor([      0,       1,       2,  ..., 1561582,  523322, 1947756])

In [17]:
srcs_tensor = srcs.to(torch.int32).cuda()
dsts_tensor = dsts.to(torch.int32).cuda()
uni = torch.ones(len(dsts)*2).to(torch.int32).cuda()

In [19]:
uni

tensor([1, 1, 1,  ..., 1, 1, 1], device='cuda:0', dtype=torch.int32)

In [20]:
srcShuffled,dstShuffled,uni = dgl.remappingNode(srcs_tensor,dsts_tensor,uni)

In [21]:
uni

tensor([      0,       1,       2,  ..., 1942208,  777620, 1285120],
       device='cuda:0', dtype=torch.int32)

In [24]:
subLabel = label[uni.to(torch.int64)]

In [25]:
subLabel.shape

torch.Size([1707527])

In [28]:
TLabel = subLabel[:len(trainBatch[0])].sum()
TLabel

tensor(300360)

: 

In [26]:
batch_size = 4
trainBatch = torch.chunk(trainIdx, batch_size, dim=0)
trainSet_1 = trainBatch[0].to(torch.int32)
trainSet_1

tensor([    0,     1,     2,  ..., 49151, 49152, 49153], dtype=torch.int32)

In [30]:
Lid = torch.zeros_like(trainSet_1).to(torch.int32).cuda()
dgl.mapLocalId(uni.cuda(),trainSet_1.cuda(),Lid)
Lid = Lid.cpu()
Lid

tensor([     1,     46,     93,  ...,  68144, 595404, 595408],
       dtype=torch.int32)

In [37]:
right = label[trainSet_1.to(torch.int64)]
right.sum()

tensor(300360)

In [36]:
error = subLabel[Lid.to(torch.int64)]
error.sum()

tensor(300143)

In [39]:
uni=uni.cpu()

In [46]:
trainBatch[0]

tensor([    0,     1,     2,  ..., 49151, 49152, 49153])

In [48]:
tt = uni[Lid.to(torch.int64)]
tt

tensor([    0,     1,     2,  ..., 49151, 49152, 49153], dtype=torch.int32)

In [64]:
tt[4677]

tensor(152857, dtype=torch.int32)

In [53]:
torch.max(tt,dim=0)

torch.return_types.max(
values=tensor(152857, dtype=torch.int32),
indices=tensor(4677))

In [57]:
sorted_indices = np.argsort(uni)

In [60]:
sorted_tensor = uni[sorted_indices]
sorted_tensor

tensor([      0,       1,       2,  ..., 2449026, 2449027, 2449028],
       dtype=torch.int32)

In [63]:
len(trainBatch[0])

49154

In [62]:
torch.max(sorted_tensor[:len(trainBatch[0])])

tensor(49169, dtype=torch.int32)

In [61]:
is_increasing = np.all(np.diff(sorted_tensor[:len(trainBatch[0])]) == 1)
is_increasing

False

In [25]:
dd = hop1.reshape(-1)[1::2]

In [24]:
nodeTable.sum()

tensor(196615)

In [7]:
for trainIdx in trainBatch:
    print(label[trainIdx].sum())

tensor(300360)
tensor(300324)
tensor(299719)
tensor(297738)


In [2]:
PATH = "/home/bear/workspace/single-gnn/data/partition/PD/processed/part"
tmpSUM = 0
for i in range(4):
    trainIds = np.fromfile(PATH + f"{i}/trainIds.bin",dtype=np.int32)
    trainIds = torch.tensor(trainIds).to(torch.int64)
    labels = np.fromfile(PATH + f"{i}/labels.bin",dtype=np.int64)
    tmplabel = labels[trainIds]
    tmpSUM = tmplabel.sum()
    print(tmpSUM)

300143
300242
299669
297596
